# Diffractometer Configuration -- Save and Restore

This document shows examples that save and restore the configuration details of
a diffractometer.  The configuration can be saved to a [file](#files).  Also,
the configuration can be saved in a bluesky [run](#run) (with the run's
metadata).  The details include:

- diffractometer name
- axes
  - real-space axis names
  - mapping to canonical real-space axes known to the solver
  - reciprocal-space axis names
  - any extra axes, as used by the solver
- sample(s)
  - lattice
  - orientation
  - reflections
- constraints
- solver
  - geometry
  - real-space axes
  - mode
  - version
- _header (date & time, energy, wavelength, source type, version)

With these configuration details, it should be possible to recover a
diffractometer orientation from a previous run or from file storage.

## Files

Create a simulated 4-circle diffractometer.  Use the factory function.  The
default will use the E4CV geometry from the hkl_soleil solver.

In [1]:
import hklpy2

sim4c = hklpy2.creator(name="sim4c")

Add a *vibranium* sample and describe its orientation.

In [2]:
import math

sim4c.add_sample("vibranium", 2*math.pi, digits=3)
sim4c.add_reflection((1, 0, 0), (10, 0, 0, 20), name="r1")
sim4c.add_reflection((0, 1, 0), (10, -90, 0, 20), name="r2")
for r in sim4c.sample.reflections.order:
    print(f"{sim4c.sample.reflections[r]}")
sim4c.operator.calc_UB(*sim4c.sample.reflections.order)

Reflection(name='r1', geometry='E4CV', pseudos={'h': 1, 'k': 0, 'l': 0}, reals={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0, digits=4)
Reflection(name='r2', geometry='E4CV', pseudos={'h': 0, 'k': 1, 'l': 0}, reals={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0, digits=4)


[[-0.0, -0.0, 1.0], [0.0, -1.0, 0.0], [1.0, -0.0, -0.0]]

Show details about the diffractometer configuration:

In [3]:
sim4c.configuration

{'_header': {'datetime': '2025-02-28 08:56:54.748993',
  'hklpy2_version': '0.0.26.dev80+g8b020e4.d20250228',
  'python_class': 'Hklpy2Diffractometer',
  'source_type': 'X-ray',
  'energy_units': 'keV',
  'energy': 12.398419843856837,
  'wavelength_units': 'angstrom',
  'wavelength': 1.0},
 'name': 'sim4c',
 'axes': {'pseudo_axes': ['h', 'k', 'l'],
  'real_axes': ['omega', 'chi', 'phi', 'tth'],
  'axes_xref': {'h': 'h',
   'k': 'k',
   'l': 'l',
   'omega': 'omega',
   'chi': 'chi',
   'phi': 'phi',
   'tth': 'tth'},
  'extra_axes': {}},
 'sample_name': 'vibranium',
 'samples': {'sample': {'name': 'sample',
   'lattice': {'a': 1,
    'b': 1,
    'c': 1,
    'alpha': 90.0,
    'beta': 90.0,
    'gamma': 90.0},
   'reflections': {},
   'reflections_order': [],
   'U': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'UB': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'digits': 4},
  'vibranium': {'name': 'vibranium',
   'lattice': {'a': 6.283185307179586,
    'b': 6.283185307179586,
    'c': 6.283185307179

### Export (save) the configuration

Save the `sim4c` configuration (with vibranium sample orientation) to a
[YAML](https://yaml.org) file in the current working directory:
`dev_e4cv-vibranium.yml`.  (Here, we choose to identify file by including the
diffractometer geometry as part of the file name.  The `dev_` prefix is an
additional arbitrary choice, you could choose a completely different file name.)

In [4]:
sim4c.export("dev_e4cv-vibranium.yml", comment="example")

Let's look at that file.  Then, compare it with the `sim4c.configuration`
reported above.

In [5]:
%pycat dev_e4cv-vibranium.yml

#hklpy2 configuration file

_header:
  datetime: '2025-02-28 08:56:54.890601'
  hklpy2_version: 0.0.26.dev80+g8b020e4.d20250228
  python_class: Hklpy2Diffractometer
  source_type: X-ray
  energy_units: keV
  energy: 12.398419843856837
  wavelength_units: angstrom
  wavelength: 1.0
  file: dev_e4cv-vibranium.yml
  comment: example
name: sim4c
axes:
  pseudo_axes:
  - h
  - k
  - l
  real_axes:
  - omega
  - chi
  - phi
  - tth
  axes_xref:
    h: h
    k: k
    l: l
    omega: omega
    chi: chi
    phi: phi
    tth: tth
  extra_axes: {}
sample_name: vibranium
samples:
  sample:
    name: sample
    lattice:
      a: 1
      b: 1
      c: 1
      alpha: 90.0
      beta: 90.0
      gamma: 90.0
    reflections: {}
    reflections_order: []
    U: &id001
    - - 1
      - 0
      - 0
    - - 0
      - 1
      - 0
    - - 0
      - 0
      - 1
    UB: *id001
    digits: 4
  vibranium:
    name: vibranium
    lattice:
      a: 6.283185307179586
      b: 6.283185307179586
      c: 6.283185307

### Restore (load) Configuration

To demonstrate restore, let's start again with an unconfigured diffractometer.
We'll just recreate the `sim4c` object.  There will only be the default `sample`.

In [6]:
sim4c = hklpy2.creator(name="sim4c")
sim4c.samples

{'sample': Sample(name='sample', lattice=Lattice(a=1, system='cubic'))}

Restore the configuration from the `dev_e4cv-vibranium.yml` file.

In [7]:
sim4c.restore("dev_e4cv-vibranium.yml", clear=True)

Show the samples again:

In [8]:
sim4c.samples

{'sample': Sample(name='sample', lattice=Lattice(a=1, system='cubic')),
 'vibranium': Sample(name='vibranium', lattice=Lattice(a=6.283, system='cubic'))}

Show the full configuration report again:

In [9]:
sim4c.configuration

{'_header': {'datetime': '2025-02-28 08:56:55.024844',
  'hklpy2_version': '0.0.26.dev80+g8b020e4.d20250228',
  'python_class': 'Hklpy2Diffractometer',
  'source_type': 'X-ray',
  'energy_units': 'keV',
  'energy': 12.398419843856837,
  'wavelength_units': 'angstrom',
  'wavelength': 1.0},
 'name': 'sim4c',
 'axes': {'pseudo_axes': ['h', 'k', 'l'],
  'real_axes': ['omega', 'chi', 'phi', 'tth'],
  'axes_xref': {'h': 'h',
   'k': 'k',
   'l': 'l',
   'omega': 'omega',
   'chi': 'chi',
   'phi': 'phi',
   'tth': 'tth'},
  'extra_axes': {}},
 'sample_name': 'vibranium',
 'samples': {'sample': {'name': 'sample',
   'lattice': {'a': 1,
    'b': 1,
    'c': 1,
    'alpha': 90.0,
    'beta': 90.0,
    'gamma': 90.0},
   'reflections': {},
   'reflections_order': [],
   'U': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'UB': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'digits': 4},
  'vibranium': {'name': 'vibranium',
   'lattice': {'a': 6.283185307179586,
    'b': 6.283185307179586,
    'c': 6.283185307179

## Run

The configuration of one or more diffractometers can be saved to the metadata of
each run.  To save the configuration(s), an instance of the
[ConfigurationRunWrapper](https://prjemian.github.io/hklpy2/api/z_misc.html)
[preprocessor](https://blueskyproject.io/bluesky/main/plans.html#supplemental-data)
is added to the `RE`.

This example uses one diffractometer, an Eulerian 4-circle named `sim4c`.

First, create a simulated 4-circle diffractometer.  We'll load the orientation
from the `dev_e4cv-vibranium.yml` file we created above.

In [10]:
import hklpy2

sim4c = hklpy2.creator(name="sim4c")
sim4c.restore("dev_e4cv-vibranium.yml", clear=True)

Setup bluesky to run standard plans with the `RunEngine`.  Also get `noisy_det`
from the ophyd simulated detectors.  To restore a diffractometer configuration
from a previous run, we'll need a catalog (`cat`) of runs.

In [11]:
from bluesky import RunEngine, plans as bp
from ophyd.sim import noisy_det
import databroker

RE = RunEngine()
cat = databroker.temp().v2
RE.subscribe(cat.v1.insert)

0

### Save configuration with every run

Create an instance of the `ConfigurationRunWrapper`, adding the `sim4c` diffractometer.

In [12]:
from hklpy2 import ConfigurationRunWrapper

crw = ConfigurationRunWrapper(sim4c)

Add the `crw` instance to the list of preprocessors that are run by the `RE`.
Each one examines the sequence of `Msg` objects internal to the RunEngine.  The
`crw.wrapper` method will add the `sim4c` diffractometer's configuration details
to every run's metadata.  (Later, we'll show how to read and restore from that
metadata.)

This configures `RE` to save `sim4c` configuration with every run.

In [13]:
RE.preprocessors.append(crw.wrapper)

Let's run a `count` plan using the `noisy_det` detector.  Keep track of the
run's `uid`, we'll use that later.

In [14]:
uids = RE(bp.count([noisy_det]))

The `sim4c` configuration is saved as a dictionary with the other metadata under
the `"diffractometers"` key.  This is a dictionary, with a key for each
diffractometer name to be reported.  Let's view the `"diffractometers"`
dictionary.  (Compare with similar views shown above.)

In [15]:
run = cat.v2[uids[0]]
run.metadata["start"]["diffractometers"]

{'sim4c': {'_header': {'datetime': '2025-02-28 08:56:55.871604',
   'hklpy2_version': '0.0.26.dev80+g8b020e4.d20250228',
   'python_class': 'Hklpy2Diffractometer',
   'source_type': 'X-ray',
   'energy_units': 'keV',
   'energy': 12.398419843856837,
   'wavelength_units': 'angstrom',
   'wavelength': 1.0},
  'name': 'sim4c',
  'axes': {'pseudo_axes': ['h', 'k', 'l'],
   'real_axes': ['omega', 'chi', 'phi', 'tth'],
   'axes_xref': {'h': 'h',
    'k': 'k',
    'l': 'l',
    'omega': 'omega',
    'chi': 'chi',
    'phi': 'phi',
    'tth': 'tth'},
   'extra_axes': {}},
  'sample_name': 'vibranium',
  'samples': {'sample': {'name': 'sample',
    'lattice': {'a': 1,
     'b': 1,
     'c': 1,
     'alpha': 90.0,
     'beta': 90.0,
     'gamma': 90.0},
    'reflections': {},
    'reflections_order': [],
    'U': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    'UB': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    'digits': 4},
   'vibranium': {'name': 'vibranium',
    'lattice': {'a': 6.283185307179586,
     'b'

### Restore from previous run

To restore from a previous run, we must have a reference to the run (with a
`uid`, `scan_id`, run object, ...).  In the steps below, we'll use the `run`
object from the previous steps.

In [16]:
run

BlueskyRun
  uid='14b377e7-b7fb-4474-b567-1234508156c7'
  exit_status='success'
  2025-02-28 08:56:55.873 -- 2025-02-28 08:56:55.898
  Streams:
    * primary


Get the configuration of the `"sim4c"` diffractometer using
`hklpy2.operations.misc.get_run_orientation()`.

In [17]:
from hklpy2.operations.misc import get_run_orientation

e4cv_configuration = get_run_orientation(run, name="sim4c")

Let's recreate `sim4c` to prove when we have recovered the configuration.

Note: This example uses the same name as before but that is not necessary.  The
configuration can be restored as long as the solver and geometry match.

In [18]:
sim4c = hklpy2.creator(name="sim4c")
sim4c.samples

{'sample': Sample(name='sample', lattice=Lattice(a=1, system='cubic'))}

Restore the configuration (from a previous run) to the `sim4c` diffractometer.
Confirm by examining the sample dictionary.

In [19]:
e4cv_configuration

{'_header': {'datetime': '2025-02-28 08:56:55.871604',
  'hklpy2_version': '0.0.26.dev80+g8b020e4.d20250228',
  'python_class': 'Hklpy2Diffractometer',
  'source_type': 'X-ray',
  'energy_units': 'keV',
  'energy': 12.398419843856837,
  'wavelength_units': 'angstrom',
  'wavelength': 1.0},
 'name': 'sim4c',
 'axes': {'pseudo_axes': ['h', 'k', 'l'],
  'real_axes': ['omega', 'chi', 'phi', 'tth'],
  'axes_xref': {'h': 'h',
   'k': 'k',
   'l': 'l',
   'omega': 'omega',
   'chi': 'chi',
   'phi': 'phi',
   'tth': 'tth'},
  'extra_axes': {}},
 'sample_name': 'vibranium',
 'samples': {'sample': {'name': 'sample',
   'lattice': {'a': 1,
    'b': 1,
    'c': 1,
    'alpha': 90.0,
    'beta': 90.0,
    'gamma': 90.0},
   'reflections': {},
   'reflections_order': [],
   'U': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'UB': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'digits': 4},
  'vibranium': {'name': 'vibranium',
   'lattice': {'a': 6.283185307179586,
    'b': 6.283185307179586,
    'c': 6.283185307179

In [20]:
sim4c.configuration = e4cv_configuration
sim4c.samples

{'sample': Sample(name='sample', lattice=Lattice(a=1, system='cubic')),
 'vibranium': Sample(name='vibranium', lattice=Lattice(a=6.283, system='cubic'))}